In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt, style; style.use('fivethirtyeight')
import seaborn as sns; sns.set(context='talk')

from IPython.display import display, HTML, IFrame

Dataset Exploration

In [ ]:
df = pd.read_csv('./data/twtc.csv')

df_label_mask = (df.drop(columns='label') == 0).assign(label=False)
df = df.mask(df_label_mask)

df.describe().T[['mean', 'std', 'min', '50%', 'max']]

In [1]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification, BertTokenizer
df = pd.read_csv('https://jacobdanovitch.blob.core.windows.net/datasets/twtc.csv')

In [7]:
df = pd.read_csv('./data/twtc.csv')
df = df.drop(['name', 'key_mlbam', 'key_fangraphs', 'age', 'year', 'primary_position', 'eta', 'Arm', 'report', 'Changeup', 'Control', 'Curveball', 'Cutter', 'Fastball', 'Field', 'Hit', 'Power', 'Run', 'Slider', 'Splitter', 'source', 'birthdate', 'mlb_played_first', 'debut_age', 'report'], axis=1)
df = df.drop(df[df['label'] == -1].index)

train, test = train_test_split(df, test_size=0.2)
custom_dataset_train = Dataset.from_pandas(train)
custom_dataset_test = Dataset.from_pandas(test)
custom_dataset_train

Dataset({
    features: ['label', 'text', '__index_level_0__'],
    num_rows: 6222
})

In [8]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [9]:
train = custom_dataset_train.map(lambda e: tokenizer(e['text'], truncation=True, padding=True), batched=True)
train = train.remove_columns(["text", "__index_level_0__"])
test = custom_dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding=True), batched=True)
test = test.remove_columns(["text", "__index_level_0__"])

In [10]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_SD_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224
)


def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=train,
    eval_dataset=test, # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
from transformers import TrainerCallback, EarlyStoppingCallback

class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=1, early_stopping_threshold=0.0))
trainer.add_callback(LoggingCallback("sample_SD_trainer/log.jsonl"))

In [ ]:
trainer.train()

/Users/amolsingh/anaconda3/envs/local_nmt/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6222
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1167


Epoch,Training Loss,Validation Loss


From_Pretrained

In [ ]:
# To load our saved model, we can pass the path to the checkpoint into the `from_pretrained` method:
test_str = "I enjoyed the movie!"

finetuned_model = AutoModelForSequenceClassification.from_pretrained("sample_SD_trainer/checkpoint-24")
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(finetuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])